# EfficientNetB0 vs MobileNet vs ResNet50V2

In [ ]:
!unzip dataset/dog-breed-identification.zip

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
labels_csv = pd.read_csv("labels.csv")
unique_labels = np.unique(labels_csv.breed)
len(unique_labels)

120

In [9]:
filenames = ['./train/' + filename + ".jpg" for filename in labels_csv.id]

In [13]:
labels = np.array(labels_csv.breed)
boolean_labels = [label == unique_labels for label in labels]

In [5]:
def resize_image(image_path):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, "float32")
  image = tf.image.resize(image, size=[224, 224])
  return image

In [6]:
def get_image_label(image_path, label):
  image = resize_image(image_path)
  return image, label

In [7]:
def create_data_batches(X, y=None, batch_size=32, valid_data=False, test_data=False):
  if test_data:
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X)))
    data_batch = data.map(resize_image).batch(32)
    return data_batch
  elif valid_data:
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
    data_batch = data.map(get_image_label).batch(32)
    return data_batch
  else:
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
    data = data.shuffle(buffer_size=len(X))
    data_batch = data.map(get_image_label).batch(32)
    return data_batch

In [14]:
X = filenames
y = boolean_labels

In [11]:
NUM_IMAGES = 10000

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X[:NUM_IMAGES], y[:NUM_IMAGES], test_size=0.2, random_state=42)

In [17]:
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

2021-12-22 19:45:21.177435: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
train_data.element_spec, val_data.element_spec

((TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 120), dtype=tf.bool, name=None)),
 (TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 120), dtype=tf.bool, name=None)))

In [19]:
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

## MobileNet50V2

In [20]:
INPUT_SHAPE = [None, 224, 224, 3]
OUTPUT_SHAPE = len(unique_labels)
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5"

In [21]:
model = tf.keras.Sequential([                  
  hub.KerasLayer(MODEL_URL),
  tf.keras.layers.Dense(OUTPUT_SHAPE, activation="softmax")
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.fit(train_data,
          epochs=100,
          validation_data=val_data,
          validation_freq=1,
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, verbose=1)])

Epoch 1/100
 22/250 [=>............................] - ETA: 2:56 - loss: 4.7004 - accuracy: 0.0980

KeyboardInterrupt: 